In [1]:
import numpy as np
import pandas as pd
import faiss

c:\Users\RS7un\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#loading vector database and csv's
#csv file contains the encodings and the laws

print("Loading FAISS index...")
train_index = faiss.read_index("language_languagearea_fact_consider.index")
print("FAISS index loaded.")

print("Loading Datasets...")
df_train= pd.read_csv("C:\\Users\\RS7un\\Desktop\\internship\\me\\all_encoded.csv")
df_test= pd.read_csv("C:\\Users\\RS7un\\Desktop\\internship\\me\\all_encoded_val.csv")
print("Datasets loaded.")



Loading FAISS index...
FAISS index loaded.
Loadig Datasets...
Datasets loaded.


In [26]:
df_train.head()

,decision_id,facts,considerations,language,law_area,laws
0,000127ef-17d2-4ded-8621-c0c962c18fd5,"[-0.01580595038831234, 0.04269539937376976, 0....","[-0.13322238624095917, 0.013987185433506966, -...","[1.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 1.0]","['75488867-c001-4eb9-93b9-04264ea91f55', 'e6b0..."
1,00015fba-e922-4f05-ae7c-7cfcb823ff54,"[-0.04270520433783531, -0.007236811798065901, ...","[-0.0926031619310379, 0.06950176507234573, 0.0...","[0.0, 1.0, 0.0]","[0.0, 0.0, 0.0, 1.0, 0.0]","['75488867-c001-4eb9-93b9-04264ea91f55', '5206..."
2,0001f593-c8af-4b97-8811-99963dfac084,"[-0.032951369881629944, 0.09559076279401779, -...","[0.0024044474121183157, -0.04070058465003967, ...","[0.0, 1.0, 0.0]","[0.0, 0.0, 0.0, 1.0, 0.0]","['75488867-c001-4eb9-93b9-04264ea91f55', '6ab3..."
3,0003a8f8-ea59-41bb-b3d6-dd58de43ef44,"[-0.098425954580307, -0.021627023816108704, -0...","[-0.06753509491682053, 0.09308979660272598, -0...","[1.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 1.0, 0.0]","['75488867-c001-4eb9-93b9-04264ea91f55', 'e6b0..."
4,00053c67-af13-4dad-baf2-f54521bfed52,"[-0.08686883002519608, 0.009472505189478397, 0...","[-0.08257345110177994, 0.019399845972657204, -...","[0.0, 1.0, 0.0]","[1.0, 0.0, 0.0, 0.0, 0.0]",[]


In [27]:
df_test.head()

,decision_id,facts,considerations,language,law_area,laws
0,00068b91-90ac-43d5-a549-9ce10728bd09,"[0.0024044474121183157, -0.04070058465003967, ...","[0.01553384866565466, -0.004896080121397972, -...","[1.0, 0.0, 0.0]","[0.0, 0.0, 1.0, 0.0, 0.0]","['75488867-c001-4eb9-93b9-04264ea91f55', '0671..."
1,001027b9-e219-4e03-9691-535a25cf3dad,"[-0.060941677540540695, 0.10897641628980637, 0...","[-0.10597439855337143, 0.016001418232917786, 0...","[1.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 1.0]","['75488867-c001-4eb9-93b9-04264ea91f55', '6ab3..."
2,00103a08-c047-4f96-9197-719b765ccc89,"[-0.08628477156162262, 0.08457666635513306, -0...","[0.0024044474121183157, -0.04070058465003967, ...","[0.0, 1.0, 0.0]","[1.0, 0.0, 0.0, 0.0, 0.0]","['75488867-c001-4eb9-93b9-04264ea91f55', '6ab3..."
3,0010fb51-807c-4ec6-aa03-48e432879443,"[-0.07059396803379059, -0.0036402500700205564,...","[-0.0867757722735405, 0.03462186083197594, -0....","[1.0, 0.0, 0.0]","[0.0, 0.0, 1.0, 0.0, 0.0]","['75488867-c001-4eb9-93b9-04264ea91f55', 'e6b0..."
4,0017573a-d123-4fa1-ad67-1b02f507438c,"[-0.10624812543392181, 0.056344181299209595, -...","[-0.059649426490068436, 0.07508216798305511, -...","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0]","['75488867-c001-4eb9-93b9-04264ea91f55', '0671..."


In [3]:
#args = (actual, retrived)
def calculate_recall(set1, set2):#how much of the correct laws are retreived
    if len(set1)==0:
        return 0
    return len(set(set1).intersection(set(set2)))/len(set1)

In [4]:
#args = (actual, retrived)
def calculate_precession(set1, set2):#how much of the laws retreived are correct
    if len(set2)==0:
        return 0
    return len(set(set1).intersection(set(set2)))/len(set2)

In [10]:
#this function returns takes test document as input and returns its encodings in vector form along with its laws

import ast
def get_index_format(df_format):
    index_format=[]
    index_format+=ast.literal_eval(df_format[3])
    index_format+=ast.literal_eval(df_format[4])
    index_format+=ast.literal_eval(df_format[2])
    index_format+=ast.literal_eval(df_format[1])
    laws=ast.literal_eval(df_format[5])
    return index_format, laws


In [30]:
recall=0
precession=0
ignored_documents=0
#can be improved by taking in count of the null lists
for k in df_test.values:
    val_index, val_laws = get_index_format(k)

    #val_index - test document in vector format
    #val_laws - test document's laws

    if len(val_laws)==0:
        ignored_documents+=1#'laws' of this test document is empty in dataset
        continue
    
    distances, indices = train_index.search(np.array([val_index]), 7)#top 7 documens in the vector db that is similar to the test document
    retreived= set()
    for i in indices[0].tolist():
        retreived.update(ast.literal_eval(df_train['laws'][i]))
    
    #'retreived' contains the laws of the top 7 documents

    if len(retreived)==0:
        ignored_documents+=1#if all the the retreived document's 'laws' are empty in dataset somehow(very rare)
        continue

    recall+= calculate_recall(val_laws, retreived)
    precession+= calculate_precession(val_laws, retreived)

average_recall = recall/(df_test.shape[0] - ignored_documents)
average_precession = precession/(df_test.shape[0] - ignored_documents)

print("Number of ignored documents: ", ignored_documents)
print("Average recall: ", average_recall)
print("Average precession: ", average_precession)

#model can perform better because,
#the existance of [] in laws of retreived documents are ignored and their penalty in evaluation parameter remains 
#the existance of none in features are ignored and encoding done with their presence

Number of ignored documents:  1926
Average recall:  0.8506810733234976
Average precession:  0.39537718636665675
